<h1>A Program for visitors map</h1>
<br>
<br>
<h3>Introduction</h3><br>
<p><b><i>AGD travels</i></b> is a company based in bangalore which provides digital maps for visitors which conatins information about the several neighbourhoods of the city. the information contains places to get food(including restaurants, cafes, hookah bars, bars ,etc.),landscapes, recreation spots, police stations and hospitals(for emergency). They also provide a cluster map which shows the neighbourhoods with similar food locations for customers with a knack of food blogging</p>

<h3>Data description</h3><br>
<p>The preliminary dataset is made from web scraping  from a wikipedia page(https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore).
    The location coordinates are searched using the geopy library of the python language.The data used in this project is provided by Foursquare location data.
    Specific Api calls are made for specific types of category. The category Id is used to get the various categorical venues. 
    The data are grouped by neighbourhood areas, and each area includes the information about restaurants, cafes, landscapes, recreation spots, police stations and hospitals present in those area.<p>





<h3>Import important libraries</h3>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import bs4
#to get the data of name of all the neighbourhoods from the wikipedia page

print('Libraries imported.')

Libraries imported.


<br>
<br>
<b>The name of the different neighbourhoods are taken from the website (<a href='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore'>List Of Neighbourhoods in Bangalore</a>) using the BeautifulSoup library.</b>

In [2]:
res=requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore')

In [3]:
soup=bs4.BeautifulSoup(res.text,'lxml')
df=[]
for i in range(8):
    for row in soup.findAll('table')[i].tbody.findAll('tr'):
        first_column = row.text.split("\n")
        df.append(first_column[1])

df=list(filter(lambda a: a != "Name", df))#removing all the empty strings

In [4]:
df

['Cantonment area',
 'Domlur',
 'Indiranagar',
 'Jeevanbheemanagar',
 'Malleswaram',
 'Pete area',
 'Sadashivanagar',
 'Seshadripuram',
 'Shivajinagar',
 'Ulsoor',
 'Vasanth Nagar',
 'Bellandur',
 'CV Raman Nagar',
 'Hoodi',
 'Krishnarajapuram',
 'Mahadevapura',
 'Marathahalli',
 'Varthur',
 'Whitefield',
 'Banaswadi',
 'HBR Layout',
 'Horamavu',
 'Kalyan Nagar',
 'Kammanahalli',
 'Lingarajapuram',
 'Ramamurthy Nagar',
 'Hebbal',
 'Jalahalli',
 'Mathikere',
 'Peenya',
 'R. T. Nagar',
 'Vidyaranyapura',
 'Yelahanka',
 'Yeshwanthpur',
 'Bommanahalli',
 'Bommasandra',
 'BTM Layout',
 'Electronic City',
 'HSR Layout',
 'Koramangala',
 'Madiwala',
 'Banashankari',
 'Basavanagudi',
 'Girinagar',
 'J. P. Nagar',
 'Jayanagar',
 'Kumaraswamy Layout',
 'Padmanabhanagar',
 'Uttarahalli',
 'Anjanapura',
 'Arekere',
 'Begur',
 'Gottigere',
 'Hulimavu',
 'Kothnur',
 'Basaveshwaranagar',
 'Kamakshipalya',
 'Kengeri',
 'Mahalakshmi Layout',
 'Nagarbhavi',
 'Nandini Layout',
 'Nayandahalli',
 'Rajajina

<br>
<br>
<b>The location coordinates of the different neighborhoods are found using the geopy library</b>
<br>
<b>This library often throws a timedOut exception so a recursive solution is used to get the cordinates. Still certain location coordinates are not found which are the removed in the cleaning portion</b>

In [5]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geolocator = Nominatim(user_agent="ny_explorer")
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)
lat=[]
long=[]


for i in range(len(df)):
    address = df[i]+',Bangalore, India '
    location = do_geocode(address)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except:
        lat.append('NaN')
        long.append('NaN')
        continue
    lat.append(latitude)
    long.append(longitude)


<h1>Cleaning the dataframe</h1>

In [6]:
df=pd.DataFrame(df)

<b>The exctracted location is added</b>

In [7]:
df["Lattitude"]=lat
df["Longitude"]=long

In [8]:
df.rename(columns={0:'Neighbourhood','Lattitude':'Latitude'},inplace=True)

In [9]:
df.columns

Index(['Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [10]:
df#the dataframe containg the name of neighbouhoods and their coordinates

,Neighbourhood,Latitude,Longitude
0,Cantonment area,13.0196,77.5096
1,Domlur,12.9625,77.6382
2,Indiranagar,12.9733,77.6405
3,Jeevanbheemanagar,NaN,NaN
4,Malleswaram,13.0163,77.5587
5,Pete area,NaN,NaN
6,Sadashivanagar,13.0074,77.5794
7,Seshadripuram,12.9932,77.5753
8,Shivajinagar,12.9864,77.6075
9,Ulsoor,12.9779,77.6247


<p><b>Removing the neighbourhoods whose coordinates are not available</b></p>

In [11]:
#removing the neighbourhoods whose coordinates are not available
for index,i in enumerate(df['Latitude']):
    if i=='NaN':
        df=df.drop(index,axis=0)

In [12]:
df_new=df.reset_index()

In [13]:
df=df_new.drop("index",axis=1)

In [14]:
df

,Neighbourhood,Latitude,Longitude
0,Cantonment area,13.0196,77.5096
1,Domlur,12.9625,77.6382
2,Indiranagar,12.9733,77.6405
3,Malleswaram,13.0163,77.5587
4,Sadashivanagar,13.0074,77.5794
5,Seshadripuram,12.9932,77.5753
6,Shivajinagar,12.9864,77.6075
7,Ulsoor,12.9779,77.6247
8,Vasanth Nagar,12.9887,77.5852
9,Bellandur,12.9791,77.5913


<h1>Define Foursquare Credentials</h1>

In [18]:
CLIENT_ID = '0Q5I0X3NMXETQOB3GECEG0RUJKGAXEJBL5SCW0VWMFY1VGMD' # your Foursquare ID
CLIENT_SECRET = 'BXY5AQACIM33CARSGFXLTAAXKEQBXWGR2P3LX4MOJD1ZSZG5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: 0Q5I0X3NMXETQOB3GECEG0RUJKGAXEJBL5SCW0VWMFY1VGMD
CLIENT_SECRET:BXY5AQACIM33CARSGFXLTAAXKEQBXWGR2P3LX4MOJD1ZSZG5


<br>
<h1>The Most Important Function</h1>
<br>
<b>This Function takes the name, lattitude, longitude, of the neighborhood and also the type of place which has to be found in the form of category ID. the list of category IDs are found from the <a href=https://developer.foursquare.com/docs/resources/categories>link</a> here. the radius of exploarion is fixed to be 500 metres</b>

In [54]:
def getNearbyVenues(names, latitudes, longitudes,categoryID,radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,categoryID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1>Search for Food(Related Places)</h1>

In [55]:
Bangalore_food_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                    categoryID='4d4b7105d754a06374d81259'
                                  )

Cantonment area
Domlur
Indiranagar
Malleswaram
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


<b>This Dataset Contains the information about various restraunts, cafes, bars etc, in the neighbourhoods of bangalore</b>

In [59]:
print(Bangalore_food_venues.shape)
Bangalore_food_venues.head(100)

(590, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cantonment area,13.019567,77.509589,New Udupi Grand,13.022775,77.509830,Vegetarian / Vegan Restaurant
1,Domlur,12.962467,77.638196,Lavonne,12.963909,77.638579,Café
2,Domlur,12.962467,77.638196,Barbeque Nation,12.962684,77.641599,BBQ Joint
3,Domlur,12.962467,77.638196,Smoke House Deli,12.965584,77.641498,Deli / Bodega
4,Domlur,12.962467,77.638196,Domino's Pizza,12.961000,77.639000,Pizza Place
5,Domlur,12.962467,77.638196,The Woodstok,12.966818,77.637182,Steakhouse
6,Domlur,12.962467,77.638196,Mainland China,12.962458,77.641727,Chinese Restaurant
7,Domlur,12.962467,77.638196,Cafe 42,12.963698,77.642039,Café
8,Domlur,12.962467,77.638196,Rajdhani,12.963834,77.641643,Rajasthani Restaurant
9,Domlur,12.962467,77.638196,Srinidhi Sagar,12.959348,77.638387,Indian Restaurant


<b>This data contains the outdoor activites present in various Neighbourhoods</b>

In [71]:
Bangalore_Outdoors_Recreation_venues=getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                    categoryID='4d4b7105d754a06377d81259'
                                  )

Cantonment area
Domlur
Indiranagar
Malleswaram
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


In [72]:
Bangalore_Outdoors_Recreation_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Domlur,12.962467,77.638196,Small public park near Domlur club,12.960969,77.639280,Park
1,Domlur,12.962467,77.638196,Total Yoga Studio,12.964770,77.639476,Yoga Studio
2,Domlur,12.962467,77.638196,akshar power yoga,12.962817,77.641539,Yoga Studio
3,Domlur,12.962467,77.638196,BDA Park,12.965599,77.636988,Park
4,Domlur,12.962467,77.638196,Cult Indiranagar,12.963619,77.641714,Gym / Fitness Center
5,Domlur,12.962467,77.638196,Ben Body Studio,12.965467,77.641594,Gym / Fitness Center
6,Indiranagar,12.973291,77.640467,Golds Gym,12.970166,77.640788,Gym
7,Indiranagar,12.973291,77.640467,Snap Fitness,12.970214,77.643403,Gym
8,Indiranagar,12.973291,77.640467,ICS Fight Club - Institute of Combat Studies,12.972903,77.644150,Martial Arts Dojo
9,Indiranagar,12.973291,77.640467,Apple Fitness Indiranagar,12.969912,77.638165,Gym / Fitness Center


<b>This data contains police stations in the city of bangalore</b>

In [77]:
Bangalore_policeStations_venues=getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                    categoryID='4bf58dd8d48988d12e941735'
                                  )

Cantonment area
Domlur
Indiranagar
Malleswaram
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


<b>Contains the Hospitals in the city</b>

In [78]:
Bangalore_Hospitals_venues=getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                    categoryID='4bf58dd8d48988d196941735'
                                  )

Cantonment area
Domlur
Indiranagar
Malleswaram
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


<h1>This function is used to generate various maps</h1>
<br>
<b>The input is the above made datasets</b>
<b>The output is the map with circle markers with labels as the names of those venues</b>

In [104]:

def gen_map(df):
    df_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)

    for lat, lng, name, categories in zip(df['Venue Latitude'], df['Venue Longitude'], 
                                               df['Venue'], df['Venue Category']):
    
        label = '{}'.format(name)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            parse_html=False).add_to(df_map)
    m=df_map
    
    return m

<h1>This map contains all the various food venues</h1>

In [109]:
gen_map(Bangalore_food_venues)

<h1>This map contains the various recreation points in the city</h1>

In [106]:
gen_map(Bangalore_Outdoors_Recreation_venues)

<h1>This map contains the location of police stations</h1>

In [108]:
gen_map(Bangalore_policeStations_venues)

<h1>This map contains the locations of Hospitals</h1>

In [110]:
gen_map(Bangalore_Hospitals_venues)

<b>Creates a dummies for all Venue Ctegories of restraunts<b>

In [57]:
# one hot encoding
food_onehot = pd.get_dummies(Bangalore_food_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
food_onehot['Neighborhood'] = Bangalore_food_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [food_onehot.columns[-1]] + list(food_onehot.columns[:-1])
food_onehot = food_onehot[fixed_columns]

food_onehot.head()

,Neighborhood,American Restaurant,Andhra Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bengali Restaurant,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Fast Food Restaurant,Food Court,Food Truck,Gastropub,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Italian Restaurant,Japanese Restaurant,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mughlai Restaurant,North Indian Restaurant,Pizza Place,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Steakhouse,Swiss Restaurant,Tex-Mex Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Domlur,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Domlur,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Domlur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Domlur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
food_grouped =food_onehot.groupby('Neighborhood').mean().reset_index()
food_grouped

,Neighborhood,American Restaurant,Andhra Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bengali Restaurant,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Fast Food Restaurant,Food Court,Food Truck,Gastropub,Hyderabadi Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Italian Restaurant,Japanese Restaurant,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mughlai Restaurant,North Indian Restaurant,Pizza Place,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Steakhouse,Swiss Restaurant,Tex-Mex Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Arekere,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.615385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,BTM Layout,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.030303,0.000000,0.121212,0.000000,0.000000,0.090909,0.000000,0.030303,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.212121,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.060606,0.000,0.060606,0.000000,0.090909,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.00
2,Banashankari,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Banaswadi,0.000000,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.00
4,Basavanagudi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000,0.000000,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Basaveshwaranagar,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.266667,0.000000,0.000000,0.066667,0.000000,0.000000,0.266667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.066667,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Begur,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [59]:
food_grouped.shape

(55, 54)

In [60]:
num_top_venues = 5

for hood in food_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = food_grouped[food_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arekere----
                venue  freq
0   Indian Restaurant  0.62
1              Bakery  0.08
2      Breakfast Spot  0.08
3  Mughlai Restaurant  0.08
4         Pizza Place  0.08


----BTM Layout----
                           venue  freq
0              Indian Restaurant  0.21
1                           Café  0.12
2  Vegetarian / Vegan Restaurant  0.09
3                         Bakery  0.09
4                    Snack Place  0.09


----Banashankari----
                             venue  freq
0                Indian Restaurant  0.25
1                       Food Truck  0.25
2  Southern / Soul Food Restaurant  0.25
3                      Pizza Place  0.25
4              American Restaurant  0.00


----Banaswadi----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.29
1              Indian Restaurant  0.29
2              Kerala Restaurant  0.14
3                      BBQ Joint  0.14
4                         Bakery  0.14


----Basavanagudi----
               

4              Snack Place  0.14


----Peenya----
                  venue  freq
0  Fast Food Restaurant   0.5
1                  Café   0.5
2   American Restaurant   0.0
3           Salad Place   0.0
4     Kerala Restaurant   0.0


----Rajajinagar----
                           venue  freq
0                    Snack Place   0.3
1              Indian Restaurant   0.2
2                 Breakfast Spot   0.2
3  Vegetarian / Vegan Restaurant   0.1
4                         Bakery   0.1


----Rajarajeshwari Nagar----
                venue  freq
0         Pizza Place  0.15
1      Breakfast Spot  0.15
2   Indian Restaurant  0.10
3      Sandwich Place  0.10
4  Chinese Restaurant  0.10


----Ramamurthy Nagar----
                 venue  freq
0    Indian Restaurant   0.4
1           Food Truck   0.4
2           Food Court   0.2
3  American Restaurant   0.0
4  Japanese Restaurant   0.0


----Sadashivanagar----
                           venue  freq
0              Indian Restaurant  0.21
1          

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = food_grouped['Neighborhood']

for ind in np.arange(food_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(food_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arekere,Indian Restaurant,Bakery,Mughlai Restaurant,Pizza Place,Breakfast Spot,Chinese Restaurant,Bengali Restaurant,Bistro,Hyderabadi Restaurant,Gastropub
1,BTM Layout,Indian Restaurant,Café,Snack Place,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Sandwich Place,Restaurant,Dim Sum Restaurant,Diner
2,Banashankari,Indian Restaurant,Food Truck,Southern / Soul Food Restaurant,Pizza Place,Chinese Restaurant,Hyderabadi Restaurant,Gastropub,Food Court,Fast Food Restaurant,Eastern European Restaurant
3,Banaswadi,Indian Restaurant,Vegetarian / Vegan Restaurant,BBQ Joint,Bakery,Kerala Restaurant,Deli / Bodega,Hyderabadi Restaurant,Gastropub,Food Truck,Food Court
4,Basavanagudi,Indian Restaurant,Restaurant,Café,Snack Place,Mediterranean Restaurant,Indian Sweet Shop,Bengali Restaurant,Bistro,Gastropub,Food Truck


In [67]:
# set number of clusters
kclusters = 5

food_grouped_clustering = food_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(food_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 3, 2, 3, 4, 3, 1, 3])

In [76]:
type(kmeans.labels_[0])

numpy.int32

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

food_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
food_merged = food_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

food_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cantonment area,13.0196,77.5096,1.0,Vegetarian / Vegan Restaurant,Wings Joint,Chettinad Restaurant,Hyderabadi Restaurant,Gastropub,Food Truck,Food Court,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
1,Domlur,12.9625,77.6382,3.0,Indian Restaurant,Café,Sandwich Place,Vietnamese Restaurant,Italian Restaurant,Deli / Bodega,Chinese Restaurant,Pizza Place,Rajasthani Restaurant,Gastropub
2,Indiranagar,12.9733,77.6405,3.0,Indian Restaurant,Café,Restaurant,Italian Restaurant,Steakhouse,Bakery,Bistro,Chinese Restaurant,Diner,Vietnamese Restaurant
3,Malleswaram,13.0163,77.5587,1.0,Vegetarian / Vegan Restaurant,Wings Joint,Chettinad Restaurant,Hyderabadi Restaurant,Gastropub,Food Truck,Food Court,Fast Food Restaurant,Eastern European Restaurant,Donut Shop
4,Sadashivanagar,13.0074,77.5794,3.0,Indian Restaurant,Café,Vegetarian / Vegan Restaurant,Snack Place,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,Pizza Place,Bakery,Chettinad Restaurant


In [87]:
food_merged['Cluster Labels'] = pd.to_numeric(food_merged['Cluster Labels'], errors='coerce')

In [88]:
food_merged = food_merged.dropna(subset=['Cluster Labels'])

In [89]:
food_merged['Cluster Labels'] = food_merged['Cluster Labels'].astype(int)

C:\Users\Anushil1998\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [90]:
# create map
map_clusters = folium.Map(location=[12.9716, 77.5946], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(food_merged['Latitude'], food_merged['Longitude'], food_merged['Neighbourhood'], food_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters